# **Exercise 3.2**



1. Create a launch that starts the **/execute_trajectory** service , called **my_robot_arm_demo.launch**. As explained in the Example 3.3, this service is launched by the launch file **start_service.launch**, which is in the package **iri_wam_reproduce_trajectory**.

2. Get information of what type of service message does this execute_trajectory service use, as explained in **Example 3.5**.

3. Make the arm robot move following a trajectory, which is specified in a file.
Modify the previous code of **Exercise 3.1**, which called the **/trajectory_by_name** service, to call now the **/execute_trajectory** service instead.

4. Here you have the code necessary to get the path to the trajectory files based on the package where they are. Here, the trajectory file **get_food.txt** is selected, but you can use any of the available in the **config** folder of the **iri_wam_reproduce_trajectory** package.

In [ ]:
#include <ros/package.h>

// This ros::package::getPath works in the same way as $(find name_of_package) in the launch files.
trajectory.request.file = ros::package::getPath("iri_wam_reproduce_trajectory") + "/config/get_food.txt";

5. In order to be able to properly compile the the **ros::package::getPath()** function, you will need to add dependencies to the **roslib** library.
 * You can do that by modifying the **find_package()** function in the **CmakeLists.txt** file. Like this:

In [ ]:
find_package(catkin REQUIRED COMPONENTS
  roscpp
  roslib
)

6. Modify the main launch file **my_robot_arm_demo.launch**, so that now it also launches the new C++ code you have just created.

7. Compile again your package.

8. Finally, execute the **my_robot_arm_demo.launch** file and see how the robot performs the trajectory.



# Solution Exercise 3.2


Step 1. create a package to place all the code.  you can call it **services_32**, with dependencies in **roscpp** and **iri_wam_reproduce_trajectory**.

In [ ]:
roscd;cd ../src
catkin_create_pkg services_32 roscpp iri_wam_reproduce_trajectory
cd services_32
mkdir launch

Step 2. Create a launch that starts the **/execute_strajectory service** , called **my_robot_arm_demo.launch**. 
 * As explained in the example 3.3, 
 > this service is launched by the launch **start_service.launch** in the package **iri_wam_reproduce_trajectory**.

**Launch File: my_robot_arm_demo.launch**



In [ ]:
<launch>

  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

  <!-- Here will go our C++ code that calls the execute_trajectory service -->

</launch>

Step 3. Launch it and check that the service **execute_trajectory** is working:



Execute in WebShell #1



In [ ]:
roslaunch services_32 my_robot_arm_demo.launch

In [ ]:
[ WARN] [1506621085.854497817, 4552.117000000]: Shutdown request received.
[ WARN] [1506621085.854552030, 4552.117000000]: Reason given for shutdown: [new node registered with same name]
[ERROR] [1506621085.965156755, 4552.207000000]: Failed to execute [get_food.txt] trajectory

Execute in WebShell #2



In [ ]:
rosservice list | grep execute_trajectory

Step 4.  Get information of what type of service message does this **execute_trajectory** service use.


Execute in WebShell #2



In [ ]:
rosservice info /execute_trajectory

In [ ]:
user:~/catkin_ws$ rosservice info /execute_trajectory
Node: /iri_wam_reproduce_trajectory
URI: rosrpc://ip-172-31-1-3:57245
Type: iri_wam_reproduce_trajectory/ExecTraj
Args: file

This tells us that if we have to call the service **execute_trajectory**, we will need to use the service message type **ExecTraj** from package **iri_wam_reproduce_trajectory**.

Execute in WebShell #2



In [ ]:
rossrv show iri_wam_reproduce_trajectory/ExecTraj

This should be the output:



In [ ]:
user:~/catkin_ws$ rossrv show iri_wam_reproduce_trajectory/ExecTraj
string file
---


1. **Request**: only a variable called file, of type String.

2. **Response**: In this case it has none. It will be what we know as Empty response.

Step 5. Now that we have all the information about the service we can start creating the C++ program to call it.

 * Make the arm robot move following a trajectory specified in a file.
 * Modify the previous code of **Exercise 3.1** that called **/trajectory_by_name**, to call the **/execute_trajectory** service instead. 
 * The new C++ file could be called **execute_trajectory.cpp** for future reference.

**C++ File: execute_trajectory.cpp**



In [ ]:
#include "ros/ros.h"
#include "iri_wam_reproduce_trajectory/ExecTraj.h"
// Import the service message used by the service /execute_trajectory

int main(int argc, char **argv)
{
  ros::init(argc, argv, "execute_trajectory_node");
  ros::NodeHandle nh;
  
                //1. Create the connection to the service /execute_trajectory
  ros::ServiceClient exec_traj_service = nh.serviceClient<iri_wam_reproduce_trajectory::ExecTraj>("/execute_trajectory");
 
                //2. Create an object of type ExecTraj
  iri_wam_reproduce_trajectory::ExecTraj trajectory; 
    
  //3. This ros::package::getPath works in the same way as $(find name_of_package) in launch files.
  trajectory.request.file = "file_path";
 
  // Fill the variable file of this object with the desired value
 

               //4.  Send through connection, name of path file to be executed   
 
  {
  if (exec_traj_service.call(trajectory)) 
    ROS_INFO("%s", "Service successfully called. Executing trajectory.");
  }
  else
  {
    ROS_ERROR("Failed to call service /execute_trajectory");
    return 1;
  }

  return 0;
}

As you can see , you need to specify the **file_path** into the **file** variable of the **Request**. But...Where are these trajectory files anyway?

* Here you have the code necessary to get the path to the **trajectory_files** based on the package where they are. Here the **trajectory_file** **get_food.txt** is selected but you can use any of the available in the **config** folder in the package **iri_wam_reproduce_trajectory**.

In [ ]:
#include <ros/package.h>

// This ros::package::getPath works in the same way as $(find name_of_package) in the launch files.
trajectory.request.file = ros::package::getPath("iri_wam_reproduce_trajectory") + "/config/get_food.txt";

Execute in WebShell #2



In [ ]:
roscd iri_wam_reproduce_trajectory
cd config
ls

In [ ]:
user:/opt/ros/indigo/share/iri_wam_reproduce_trajectory/config$ ls
get_food.txt  init_pos.txt  release_food.txt

Step 6. Copt the path to config 

In [ ]:
roscd services_32
mkdir config
roscd iri_wam_reproduce_trajectory/config
cp get_food.txt /home/user/catkin_ws/src/services_32/config

In [ ]:
0.0112138,0.942628,0.133408,1.65916,-0.214736,-1.05983,-0.430974,0.00213521,0.313696,0.0892853,-0.29252,-0.110414,
0.0677202,0.140693,-0.00383239,-0.0665293,0.0935825,-0.27529,-0.393899,0.351521,-0.0444471,0.579301,2.77829,
-2.87128,0.0812354,-0.336139,-0.000311573,0.0183247,-1.86726,1.48766,3.67008,-1.55314,0.302464,0.355123,0.0429483,0.0256781,
-0.100288,0.0139974,-0.00673008,-0.00101568,-0.000844474,-1.56452e-05,-0.687394,-2.01554,2.78754,0.84521,-0.523724,0.667854,0.0283819


Step 7. update the trajectory with the file of poses

**C++ File: Update file_execute_trajectory.cpp**



In [ ]:
#include "ros/ros.h"
#include "iri_wam_reproduce_trajectory/ExecTraj.h"
#include <ros/package.h>
// Import rospackage

int main(int argc, char **argv)
{
  ros::init(argc, argv, "execute_trajectory_node");
  ros::NodeHandle nh;
  
                   // 1. Create the connection to the service /execute_trajectory
  ros::ServiceClient exec_traj_service = nh.serviceClient<iri_wam_reproduce_trajectory::ExecTraj>("/execute_trajectory");
 
                 // 2. Create an object of type ExecTraj
  iri_wam_reproduce_trajectory::ExecTraj trajectory; 
    
  //3.  This ros::package::getPath works in the same way as $(find name_of_package) in the launch files.
  trajectory.request.file = ros::package::getPath("iri_wam_reproduce_trajectory") + "/config/get_food.txt";
  // Fill the variable file of this object with the desired value
  
  if (exec_traj_service.call(trajectory)) 
  {
    ROS_INFO("%s", "Service successfully called. Executing trajectory.");
  }
  else
  {
    ROS_ERROR("Failed to call service /execute_trajectory");
    return 1;
  }

  return 0;
}

**CMaleList.txt**

In [ ]:
find_package(catkin REQUIRED COMPONENTS
  roscpp
  roslib
  iri_wam_reproduce_trajectory
)

###################################
## catkin specific configuration ##
###################################

catkin_package(
  INCLUDE_DIRS include
 #  LIBRARIES services_32
  CATKIN_DEPENDS iri_wam_reproduce_trajectory roscpp roslib
#  DEPENDS system_lib
)

###########
## Build ##
###########

include_directories(
# include
  ${catkin_INCLUDE_DIRS}
)


add_executable(file_execute_trajectory src/file_execute_trajectory.cpp)

add_dependencies(file_execute_trajectory 
      ${file_execute_trajectory_EXPORTED_TARGETS} ${catkin_EXPORTED_TARGETS})

target_link_libraries(file_execute_trajectory 
   ${catkin_LIBRARIES}
 )



**packge.xml**

In [ ]:
  <build_depend>iri_wam_reproduce_trajectory</build_depend>
  <build_export_depend>iri_wam_reproduce_trajectory</build_export_depend>
  <exec_depend>iri_wam_reproduce_trajectory</exec_depend>

  <build_depend>roslib</build_depend>
  <exec_depend>roslib</exec_depend>
  <build_export_depend>roslib</build_export_depend>

Step 8.Add the call of **execute_trajectory.cpp** to the main launch **my_robot_arm_demo.launch.**


In [ ]:
<launch>

  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

  <!-- Here will go our C++ code that calls the execute_trajectory service -->
    <node pkg ="services_32"
        type="file_execute_trajectory"
        name="execute_trajectory_node"
        output="screen">
  </node>
  
</launch>

Now Launch the **my_robot_arm_demo.launch** and see how the robot performs the trajectory.


You should get in case of executeing the **trajectory get_food.txt**, the following movement:

